# Week 4: Scene Flow Estimation

If you are running on Colab,
* Go to runtime -> change runtime type -> select "GPU" as the hardware accelerator. 

This week's assignment has two parts. 

## Part A

In the first part, you'll annotate the code below for [FlowNet3D](https://arxiv.org/abs/1806.01411). What does it mean to annotate a notebook?

The original implementation of FlowNet3D is in TensorFlow: [tf-code](https://github.com/xingyul/flownet3d). Someone has implemented it in PyTorch and provided the notebook below: [pytorch-code](https://github.com/multimodallearning/flownet3d.pytorch). This is great but the notebook is not very helpful for someone trying to match it to the paper.

When it is done right, it looks like this: [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html). Can you provide a similar encapsulation for the code below by putting relevant parts of the paper before the code segment? Feel free to divide code segments further and copy text or figures from the paper but make sure that it matches the code and furthermore explains what is done in the code.

# 3. Problem Definition
<img align="right" width="400" height="200" src="https://drive.google.com/uc?id=1i0LZbqrfOnsCYJLbHBrfAPSARPD9HRUV">


Input to the network are two sets of points sampled from a dynamic 3D scene, at two consecutive time frames: $\mathcal{P}=\left\{x_{i} \mid i=\right.$ $\left.1, \ldots, n_{1}\right\}($ point cloud 1$)$ and $\mathcal{Q}=\left\{y_{j} \mid j=1, \ldots, n_{2}\right\}$
(point cloud 2), where $x_{i}, y_{j} \in \mathbb{R}^{3}$ are $X Y Z$ coordinates of individual points.

Now consider the physical point under a sampled point $x_{i}$ moves to location $x_{i}^{\prime}$ at the second frame, then the translational motion vector of the point is $d_{i}=x_{i}^{\prime}-x_{i} .$ The goal is, given $\mathcal{P}$ and $\mathcal{Q},$ to recover the scene flow for every sampled point in the first frame: $\mathcal{D}=\left\{d_{i} \mid i=1, \ldots, n_{1}\right\}$.



# 4. FlowNet3D Architecture
The model has three key modules for (1) point feature learning,
(2) point mixture, and (3) flow refinement.

## 4.1. Hierarchical Point Cloud Feature Learning
<img align="right" width="500" height="200" src="https://drive.google.com/uc?id=1vzPt7Y3DJl6to_HlJEGDNH37aOI8CCu8">

set conv layer is used to learn deep point cloud features. As shown in the figure, a set conv layer takes a point cloud with $n$ points, each point $p_{i}=\left\{x_{i}, f_{i}\right\}$ with its $X Y Z$ coordinates $x_{i} \in \mathbb{R}^{3}$ and its feature $f_{i} \in \mathbb{R}^{c}(i=1, \ldots, n)$
and outputs a sub-sampled point cloud with $n^{\prime}$ points, where each point $p_{j}^{\prime}=\left\{x_{j}^{\prime}, f_{j}^{\prime}\right\}$ has its $X Y Z$ coordinates $x_{j}^{\prime}$ and an updated point feature $f_{j}^{\prime} \in \mathbb{R}^{c^{\prime}}\left(j=1, \ldots n^{\prime}\right)$

the layer firstly samples $n'$ regions from the input points with farthest point sampling (with region centers as $x'_j$):

```
class Sample(nn.Module):
    def __init__(self, num_points):
        super(Sample, self).__init__()
        self.num_points = num_points
        
    def forward(self, points):
        new_points_ind = furthest_point_sampling(points.permute(0, 2, 1).contiguous(), self.num_points)
        new_points = fps_gather_by_index(points, new_points_ind)
        return new_points
```

Then for each region (defined by a radius neighborhood specified by radius $r)$, it extracts its local feature with the following symmetric function
$$
f_{j}^{\prime}=\operatorname{MAX}_{\left\{i \mid\left\|x_{i}-x_{j}^{\prime}\right\| \leq r\right\}}\left\{h\left(f_{i}, x_{i}-x_{j}^{\prime}\right)\right\}
$$
where $h: \mathbb{R}^{c+3} \rightarrow \mathbb{R}^{c^{\prime}}$ is a non-linear function (realized as a multi-layer perceptron) with concatenated $f_{i}$ and $x_{i}-x_{j}^{\prime}$ as inputs, and MAX is element-wise max pooling.

```
class Group(nn.Module):
    def __init__(self, radius, num_samples, knn=False):
        super(Group, self).__init__()
        
        self.radius = radius
        self.num_samples = num_samples
        self.knn = knn
        
    def forward(self, points, new_points, features):
        if self.knn:
            dist = pdist2squared(points, new_points)
            ind = dist.topk(self.num_samples, dim=1, largest=False)[1].int().permute(0, 2, 1).contiguous()
        else:
            ind = ball_query(self.radius, self.num_samples, points.permute(0, 2, 1).contiguous(),
                             new_points.permute(0, 2, 1).contiguous(), False)
        grouped_points = group_gather_by_index(points, ind)
        grouped_points -= new_points.unsqueeze(3)
        grouped_features = group_gather_by_index(features, ind)
        new_features = torch.cat([grouped_points, grouped_features], dim=1)
        return new_features

class SetConv(nn.Module):
    def __init__(self, num_points, radius, num_samples, in_channels, out_channels):
        super(SetConv, self).__init__()
        
        self.sample = Sample(num_points)
        self.group = Group(radius, num_samples)
        
        layers = []
        out_channels = [in_channels+3, *out_channels]
        for i in range(1, len(out_channels)):
            layers += [nn.Conv2d(out_channels[i - 1], out_channels[i], 1, bias=True), nn.BatchNorm2d(out_channels[i], eps=0.001), nn.ReLU()]
        self.conv = nn.Sequential(*layers)
        
    def forward(self, points, features):
        new_points = self.sample(points)
        new_features = self.group(points, new_points, features)
        new_features = self.conv(new_features)
        new_features = new_features.max(dim=3)[0]
        return new_points, new_features
```

## 4.2. Point Mixture with Flow Embedding Layer

<img align="right" width="400" height="200" src="https://drive.google.com/uc?id=1NtBZ9xt-gS0HZ4hNrzDtt3jF6HGsaACk">

To mix two point clouds they rely on a new flow embedding
layer shown in the figure. The layer learns a flow embedding
for each point in the first frame. the flow embedding layer takes a pair of point clouds: $\left\{p_{i}=\left(x_{i}, f_{i}\right)\right\}_{i=1}^{n_{1}}$ and $\left\{q_{j}=\left(y_{j}, g_{j}\right)\right\}_{j=1}^{n_{2}}$ where
each point has its $X Y Z$ coordinate $x_{i}, y_{j} \in \mathbb{R}^{3},$ and a feature vector $f_{i}, g_{j} \in \mathbb{R}^{c} .$ 
They use a neural layer to aggregate flow votes from
all the neighboring,

$$
e_{i}=\operatorname{MAX}_{\left\{j \mid\left\|_{j}-x_{i}\right\| \leq r\right\}}\left\{h\left(f_{i}, g_{j}, y_{j}-x_{i}\right)\right\}
$$

```
class FlowEmbedding(nn.Module):
    def __init__(self, num_samples, in_channels, out_channels):
        super(FlowEmbedding, self).__init__()
        
        self.num_samples = num_samples
        
        self.group = Group(None, self.num_samples, knn=True)
        
        layers = []
        out_channels = [2*in_channels+3, *out_channels]
        for i in range(1, len(out_channels)):
            layers += [nn.Conv2d(out_channels[i - 1], out_channels[i], 1, bias=True), nn.BatchNorm2d(out_channels[i], eps=0.001), nn.ReLU()]
        self.conv = nn.Sequential(*layers)
        
    def forward(self, points1, points2, features1, features2):
        new_features = self.group(points2, points1, features2)
        new_features = torch.cat([new_features, features1.unsqueeze(3).expand(-1, -1, -1, self.num_samples)], dim=1)
        new_features = self.conv(new_features)
        new_features = new_features.max(dim=3)[0]
        return new_features
```

## 4.3. Flow Refinement with Set Upconv Layer
<img align="right" width="500" height="200" src="https://drive.google.com/uc?id=1HV-QpBJsBzYVEbbtSSu8kIC5jfIbtG1S">
The up-sampling step is achieved by a learnable new layer
– the set upconv layer, which learns to propagate and refine
the embeddings in an informed way. The inputs to the layer are source points $\left\{p_{i}=\right.$ $\left.\left\{x_{i}, f_{i}\right\} \mid i=1, \ldots, n\right\},$ and a set of target point coordinates $\left\{x_{j}^{\prime} \mid j=1, \ldots, n^{\prime}\right\}$ which are locations we want to propagate the source point features to.

Interestingly, just like in 2D convolutions in images
where upconv2D can be implemented through conv2D, our
set upconv can also be directly achieved with the same set
conv layer as defined in Eq. (1), but with a different local region
sampling strategy. Instead of using farthest point sampling, they compute features on specified locations by the target points.

```
class SetUpConv(nn.Module):
    def __init__(self, num_samples, in_channels1, in_channels2, out_channels1, out_channels2):
        super(SetUpConv, self).__init__()
        
        self.group = Group(None, num_samples, knn=True)
        
        layers = []
        out_channels1 = [in_channels1+3, *out_channels1]
        for i in range(1, len(out_channels1)):
            layers += [nn.Conv2d(out_channels1[i - 1], out_channels1[i], 1, bias=True), nn.BatchNorm2d(out_channels1[i], eps=0.001), nn.ReLU()]
        self.conv1 = nn.Sequential(*layers)
        
        layers = []
        if len(out_channels1) == 1:
            out_channels2 = [in_channels1+in_channels2+3, *out_channels2]
        else:
            out_channels2 = [out_channels1[-1]+in_channels2, *out_channels2]
        for i in range(1, len(out_channels2)):
            layers += [nn.Conv2d(out_channels2[i - 1], out_channels2[i], 1, bias=True), nn.BatchNorm2d(out_channels2[i], eps=0.001), nn.ReLU()]
        self.conv2 = nn.Sequential(*layers)
        
    def forward(self, points1, points2, features1, features2):
        new_features = self.group(points1, points2, features1)
        new_features = self.conv1(new_features)
        new_features = new_features.max(dim=3)[0]
        new_features = torch.cat([new_features, features2], dim=1)
        new_features = new_features.unsqueeze(3)
        new_features = self.conv2(new_features)
        new_features = new_features.squeeze(3)
        return new_features

class FeaturePropagation(nn.Module):
    def __init__(self, in_channels1, in_channels2, out_channels):
        super(FeaturePropagation, self).__init__()
        
        layers = []
        out_channels = [in_channels1+in_channels2, *out_channels]
        for i in range(1, len(out_channels)):
            layers += [nn.Conv2d(out_channels[i - 1], out_channels[i], 1, bias=True), nn.BatchNorm2d(out_channels[i], eps=0.001), nn.ReLU()]
        self.conv = nn.Sequential(*layers)
        
    def forward(self, points1, points2, features1, features2):
        dist, ind = three_nn(points2.permute(0, 2, 1).contiguous(), points1.permute(0, 2, 1).contiguous())
        dist = dist * dist
        dist[dist < 1e-10] = 1e-10
        inverse_dist = 1.0 / dist
        norm = torch.sum(inverse_dist, dim=2, keepdim=True)
        weights = inverse_dist / norm
        #new_features = three_interpolate(features1, ind, weights) # wrong gradients
        new_features = torch.sum(group_gather_by_index(features1, ind) * weights.unsqueeze(1), dim = 3)
        new_features = torch.cat([new_features, features2], dim=1)
        new_features = self.conv(new_features.unsqueeze(3)).squeeze(3)
        return new_features
```



## 4.4. Network Architecture

The final FlowNet3D architecture is composed of four
set conv layers, one flow embedding layer and four set upconv
layers (corresponding to the four set conv layers) and
a final linear flow regression layer that outputs the R3 predicted
scene flow.

<img align="center" width="800" height="300" src="https://drive.google.com/uc?id=1QfeVo0GvSmt4xiBet9sPEvsSI_rxjQIG">

```
class FlowNet3D(nn.Module):
    def __init__(self):
        super(FlowNet3D, self).__init__()
        
        self.set_conv1 = SetConv(1024, 0.5, 16, 3, [32, 32, 64])
        self.set_conv2 = SetConv(256, 1.0, 16, 64, [64, 64, 128])
        self.flow_embedding = FlowEmbedding(64, 128, [128, 128, 128])
        self.set_conv3 = SetConv(64, 2.0, 8, 128, [128, 128, 256])
        self.set_conv4 = SetConv(16, 4.0, 8, 256, [256, 256, 512])
        self.set_upconv1 = SetUpConv(8, 512, 256, [], [256, 256])
        self.set_upconv2 = SetUpConv(8, 256, 256, [128, 128, 256], [256])
        self.set_upconv3 = SetUpConv(8, 256, 64, [128, 128, 256], [256])
        self.fp = FeaturePropagation(256, 3, [256, 256])
        self.classifier = nn.Sequential(
            nn.Conv1d(256, 128, 1, bias=True),
            nn.BatchNorm1d(128, eps=0.001),
            nn.ReLU(),
            nn.Conv1d(128, 3, 1, bias=True)
        )
         
    def forward(self, points1, points2, features1, features2):
        points1_1, features1_1 = self.set_conv1(points1, features1)
        points1_2, features1_2 = self.set_conv2(points1_1, features1_1)

        points2_1, features2_1 = self.set_conv1(points2, features2)
        points2_2, features2_2 = self.set_conv2(points2_1, features2_1)

        embedding = self.flow_embedding(points1_2, points2_2, features1_2, features2_2)
        
        points1_3, features1_3 = self.set_conv3(points1_2, embedding)
        points1_4, features1_4 = self.set_conv4(points1_3, features1_3)
        
        new_features1_3 = self.set_upconv1(points1_4, points1_3, features1_4, features1_3)
        new_features1_2 = self.set_upconv2(points1_3, points1_2, new_features1_3, torch.cat([features1_2, embedding], dim=1))
        new_features1_1 = self.set_upconv3(points1_2, points1_1, new_features1_2, features1_1)
        new_features1 = self.fp(points1_1, points1, new_features1_1, features1)

        flow = self.classifier(new_features1)
        
        return flow
```



## Training loss
They use smooth L1 loss (huber loss) for scene flow supervision,
together with a cycle-consistency regularization.

Given a point cloud $\mathcal{P}=\left\{x_{i}\right\}_{i=1}^{n_{1}}$ at frame $t$ and a point cloud $\mathcal{Q}=\left\{y_{j}\right\}_{j=1}^{n_{2}}$ at frame $t+1,$ the network predicts scene flow as $\mathcal{D}=F(\mathcal{P}, \mathcal{Q} ; \Theta)=\left\{d_{i}\right\}_{i=1}^{n_{1}}$ where $F$ is the
FlowNet3D model with parameters $\Theta$. With ground truth scene flow $\mathcal{D}^{*}=\left\{d_{i}^{*}\right\}_{i=1}^{n_{1}},$ our loss is defined as shown here. In the equation, $\left\|d_{i}^{\prime}+d_{i}\right\|$ is the cycle-consistency term that enforces the backward flow $\left\{d_{i}^{\prime}\right\}_{i=1}^{n_{1}}=F\left(\mathcal{P}^{\prime}, \mathcal{P} ; \Theta\right)$ from
the shifted point cloud $\mathcal{P}^{\prime}=\left\{x_{i}+d_{i}\right\}_{i=1}^{n_{1}}$ to the original point cloud $\mathcal{P}$ is close to the reverse of the forward flow

$$
L\left(\mathcal{P}, \mathcal{Q}, \mathcal{D}^{*}, \Theta\right)=\frac{1}{n_{1}} \sum_{i=1}^{n_{1}}\left\{\left\|d_{i}-d_{i}^{*}\right\|+\lambda\left\|d_{i}^{\prime}+d_{i}\right\|\right\}
$$

But the implemented loss is just a L2 loss:

```
def criterion(pred_flow, flow, mask):
    loss = torch.mean(mask * torch.sum((pred_flow - flow) * (pred_flow - flow), dim=1) / 2.0)
    return loss
```

## Part B

In the second part, you will extend the code to KITTI. You can use the models pre-trained on FlyingThings3D in the `models` folder. For KITTI, please use the preprocessed data by removing ground from the original TensorFlow repo:

"We release the processed KITTI scene flow dataset [here](https://drive.google.com/open?id=1XBsF35wKY0rmaL7x7grD_evvKCAccbKi) for download (total size ~266MB). The KITTI scene flow dataset was processed by converting the 2D optical flow into 3D scene flow and removing the ground points. We processed the first 150 data points from KITTI scene flow dataset. Each of the data points are stored as a .npz file and its dictionary has three keys: pos1, pos2 and gt, representing the first frame of point cloud, second frame of point cloud and the ground truth scene flow vectors for the points in the first frame."

**Question**: Using the model trained on FlyingThings3D and evaluating it on the pre-processed KITTI, can you reproduce the results in the last row of the Table 4? You will need to implement the 'outliers' metric

**Answer**: We have found 2% difference in EPE and 6% difference in the outlier metric. The two main reasons are the difference in the loss function and the lack of colored features in the dataset.

### Hints
For running the code you will need to install kaolin v0.1.
You can use the following lines to install Kaolin on Google Colab, or you can follow the [installation guide](https://kaolin.readthedocs.io/en/latest/notes/installation.html).

In [ ]:
!git clone -b v0.1 https://github.com/NVIDIAGameWorks/kaolin.git

fatal: destination path 'kaolin' already exists and is not an empty directory.


In [ ]:
# !pip install kaolin/

The installation might take a couple of minutes. After it is done, restart the runtime on Google on Colab. After restarting, Kaolin should be installed correctly.

You will also need to the [flownet3d.pytorch](https://github.com/multimodallearning/flownet3d.pytorch) repository, which you can clone:

In [ ]:
!git clone https://github.com/multimodallearning/flownet3d.pytorch.git

fatal: destination path 'flownet3d.pytorch' already exists and is not an empty directory.


You can use `gdown` for downloading the [KITTI data](https://drive.google.com/open?id=1XBsF35wKY0rmaL7x7grD_evvKCAccbKi):

In [ ]:
!gdown --id 1XBsF35wKY0rmaL7x7grD_evvKCAccbKi

Downloading...
From: https://drive.google.com/uc?id=1XBsF35wKY0rmaL7x7grD_evvKCAccbKi
To: /content/kitti_rm_ground.tar
279MB [00:02, 99.0MB/s]


In [ ]:
!tar -xvf kitti_rm_ground.tar

kitti_rm_ground/
kitti_rm_ground/000054.npz
kitti_rm_ground/000080.npz
kitti_rm_ground/000091.npz
kitti_rm_ground/000147.npz
kitti_rm_ground/000024.npz
kitti_rm_ground/000104.npz
kitti_rm_ground/000035.npz
kitti_rm_ground/000144.npz
kitti_rm_ground/000119.npz
kitti_rm_ground/000026.npz
kitti_rm_ground/000051.npz
kitti_rm_ground/000131.npz
kitti_rm_ground/000052.npz
kitti_rm_ground/000095.npz
kitti_rm_ground/000060.npz
kitti_rm_ground/000084.npz
kitti_rm_ground/000077.npz
kitti_rm_ground/000089.npz
kitti_rm_ground/000048.npz
kitti_rm_ground/000097.npz
kitti_rm_ground/000120.npz
kitti_rm_ground/000082.npz
kitti_rm_ground/000129.npz
kitti_rm_ground/000002.npz
kitti_rm_ground/000088.npz
kitti_rm_ground/000010.npz
kitti_rm_ground/000116.npz
kitti_rm_ground/000006.npz
kitti_rm_ground/000014.npz
kitti_rm_ground/000069.npz
kitti_rm_ground/000071.npz
kitti_rm_ground/000105.npz
kitti_rm_ground/000058.npz
kitti_rm_ground/000008.npz
kitti_rm_ground/000124.npz
kitti_rm_ground/000036.npz
kitti_rm_gr

Now you can try extending and running the code for KITTI.

In [ ]:
import glob
import json
import matplotlib.pyplot as plt
from multiprocessing import Manager
import numpy as np
import os
import time
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import kaolin as kal
from kaolin.models.PointNet2 import furthest_point_sampling
from kaolin.models.PointNet2 import fps_gather_by_index
from kaolin.models.PointNet2 import ball_query
from kaolin.models.PointNet2 import group_gather_by_index
from kaolin.models.PointNet2 import three_nn
from kaolin.models.PointNet2 import three_interpolate

import tensorflow as tf

/usr/local/lib/python3.7/dist-packages/kaolin/datasets/__init__.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado

   No module named 'nuscenes'
   None


In [ ]:
class TestDataset(Dataset):
    def __init__(self):
      self.dir = './kitti_rm_ground/'
      self.files = os.listdir(self.dir)

    def __getitem__(self, index, npoints=2048):
      data = np.load(self.dir + self.files[index])
      pos1 = data['pos1'].astype('float32')
      pos2 = data['pos2'].astype('float32')
      flow = data['gt'].astype('float32')

      pos1_center = np.mean(pos1, 0)
      pos1 -= pos1_center
      pos2 -= pos1_center
        
      pos1 = torch.from_numpy(pos1[:npoints]).t()
      pos2 = torch.from_numpy(pos2[:npoints]).t()
      flow = torch.from_numpy(flow[:npoints]).t()

      return pos1, pos2, flow

    def __len__(self):
        return len(self.files)
    
test_set = TestDataset()
points1, points2, flow = test_set[0]

print(points1.shape, points1.dtype)
print(points2.shape, points2.dtype)
print(flow.shape, flow.dtype)

torch.Size([3, 2048]) torch.float32
torch.Size([3, 2048]) torch.float32
torch.Size([3, 2048]) torch.float32


In [ ]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

def pdist2squared(x, y):
    xx = (x**2).sum(dim=1).unsqueeze(2)
    yy = (y**2).sum(dim=1).unsqueeze(1)
    dist = xx + yy - 2.0 * torch.bmm(x.permute(0, 2, 1), y)
    dist[dist != dist] = 0
    dist = torch.clamp(dist, 0.0, np.inf)
    return dist

def parameter_count(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class ClippedStepLR(optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, step_size, min_lr, gamma=0.1, last_epoch=-1):
        self.step_size = step_size
        self.min_lr = min_lr
        self.gamma = gamma
        super(ClippedStepLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [max(base_lr * self.gamma ** (self.last_epoch // self.step_size), self.min_lr)
                for base_lr in self.base_lrs]
    
def criterion(pred_flow, flow, mask):
    loss = torch.mean(mask * torch.sum((pred_flow - flow) * (pred_flow - flow), dim=1) / 2.0)
    return loss

def error(pred, labels, mask):
    pred = pred.permute(0,2,1).cpu().numpy()
    labels = labels.permute(0,2,1).cpu().numpy()
    mask = mask.cpu().numpy()
    
    err = np.sqrt(np.sum((pred - labels)**2, 2) + 1e-20)

    gtflow_len = np.sqrt(np.sum(labels*labels, 2) + 1e-20) # B,N
    acc050 = np.sum(np.logical_or((err <= 0.05)*mask, (err/gtflow_len <= 0.05)*mask), axis=1)
    acc010 = np.sum(np.logical_or((err <= 0.1)*mask, (err/gtflow_len <= 0.1)*mask), axis=1)

    mask_sum = np.sum(mask, 1)
    acc050 = acc050[mask_sum > 0] / mask_sum[mask_sum > 0]
    acc050 = np.mean(acc050)
    acc010 = acc010[mask_sum > 0] / mask_sum[mask_sum > 0]
    acc010 = np.mean(acc010)

    epe = np.sum(err * mask, 1)[mask_sum > 0] / mask_sum[mask_sum > 0]
    epe = np.mean(epe)
    return epe, acc050, acc010

In [ ]:
from kaolin.models.PointNet2 import furthest_point_sampling
from kaolin.models.PointNet2 import fps_gather_by_index

class Sample(nn.Module):
    def __init__(self, num_points):
        super(Sample, self).__init__()
        
        self.num_points = num_points
        
    def forward(self, points):
        new_points_ind = furthest_point_sampling(points.permute(0, 2, 1).contiguous(), self.num_points)
        new_points = fps_gather_by_index(points, new_points_ind)
        return new_points
    
class Group(nn.Module):
    def __init__(self, radius, num_samples, knn=False):
        super(Group, self).__init__()
        
        self.radius = radius
        self.num_samples = num_samples
        self.knn = knn
        
    def forward(self, points, new_points, features):
        if self.knn:
            dist = pdist2squared(points, new_points)
            ind = dist.topk(self.num_samples, dim=1, largest=False)[1].int().permute(0, 2, 1).contiguous()
        else:
            ind = ball_query(self.radius, self.num_samples, points.permute(0, 2, 1).contiguous(),
                             new_points.permute(0, 2, 1).contiguous(), False)
        grouped_points = group_gather_by_index(points, ind)
        grouped_points -= new_points.unsqueeze(3)
        grouped_features = group_gather_by_index(features, ind)
        new_features = torch.cat([grouped_points, grouped_features], dim=1)
        return new_features

class SetConv(nn.Module):
    def __init__(self, num_points, radius, num_samples, in_channels, out_channels):
        super(SetConv, self).__init__()
        
        self.sample = Sample(num_points)
        self.group = Group(radius, num_samples)
        
        layers = []
        out_channels = [in_channels+3, *out_channels]
        for i in range(1, len(out_channels)):
            layers += [nn.Conv2d(out_channels[i - 1], out_channels[i], 1, bias=True), nn.BatchNorm2d(out_channels[i], eps=0.001), nn.ReLU()]
        self.conv = nn.Sequential(*layers)
        
    def forward(self, points, features):
        new_points = self.sample(points)
        new_features = self.group(points, new_points, features)
        new_features = self.conv(new_features)
        new_features = new_features.max(dim=3)[0]
        return new_points, new_features
    
class FlowEmbedding(nn.Module):
    def __init__(self, num_samples, in_channels, out_channels):
        super(FlowEmbedding, self).__init__()
        
        self.num_samples = num_samples
        
        self.group = Group(None, self.num_samples, knn=True)
        
        layers = []
        out_channels = [2*in_channels+3, *out_channels]
        for i in range(1, len(out_channels)):
            layers += [nn.Conv2d(out_channels[i - 1], out_channels[i], 1, bias=True), nn.BatchNorm2d(out_channels[i], eps=0.001), nn.ReLU()]
        self.conv = nn.Sequential(*layers)
        
    def forward(self, points1, points2, features1, features2):
        new_features = self.group(points2, points1, features2)
        new_features = torch.cat([new_features, features1.unsqueeze(3).expand(-1, -1, -1, self.num_samples)], dim=1)
        new_features = self.conv(new_features)
        new_features = new_features.max(dim=3)[0]
        return new_features
    
class SetUpConv(nn.Module):
    def __init__(self, num_samples, in_channels1, in_channels2, out_channels1, out_channels2):
        super(SetUpConv, self).__init__()
        
        self.group = Group(None, num_samples, knn=True)
        
        layers = []
        out_channels1 = [in_channels1+3, *out_channels1]
        for i in range(1, len(out_channels1)):
            layers += [nn.Conv2d(out_channels1[i - 1], out_channels1[i], 1, bias=True), nn.BatchNorm2d(out_channels1[i], eps=0.001), nn.ReLU()]
        self.conv1 = nn.Sequential(*layers)
        
        layers = []
        if len(out_channels1) == 1:
            out_channels2 = [in_channels1+in_channels2+3, *out_channels2]
        else:
            out_channels2 = [out_channels1[-1]+in_channels2, *out_channels2]
        for i in range(1, len(out_channels2)):
            layers += [nn.Conv2d(out_channels2[i - 1], out_channels2[i], 1, bias=True), nn.BatchNorm2d(out_channels2[i], eps=0.001), nn.ReLU()]
        self.conv2 = nn.Sequential(*layers)
        
    def forward(self, points1, points2, features1, features2):
        new_features = self.group(points1, points2, features1)
        new_features = self.conv1(new_features)
        new_features = new_features.max(dim=3)[0]
        new_features = torch.cat([new_features, features2], dim=1)
        new_features = new_features.unsqueeze(3)
        new_features = self.conv2(new_features)
        new_features = new_features.squeeze(3)
        return new_features
    
class FeaturePropagation(nn.Module):
    def __init__(self, in_channels1, in_channels2, out_channels):
        super(FeaturePropagation, self).__init__()
        
        layers = []
        out_channels = [in_channels1+in_channels2, *out_channels]
        for i in range(1, len(out_channels)):
            layers += [nn.Conv2d(out_channels[i - 1], out_channels[i], 1, bias=True), nn.BatchNorm2d(out_channels[i], eps=0.001), nn.ReLU()]
        self.conv = nn.Sequential(*layers)
        
    def forward(self, points1, points2, features1, features2):
        dist, ind = three_nn(points2.permute(0, 2, 1).contiguous(), points1.permute(0, 2, 1).contiguous())
        dist = dist * dist
        dist[dist < 1e-10] = 1e-10
        inverse_dist = 1.0 / dist
        norm = torch.sum(inverse_dist, dim=2, keepdim=True)
        weights = inverse_dist / norm
        #new_features = three_interpolate(features1, ind, weights) # wrong gradients
        new_features = torch.sum(group_gather_by_index(features1, ind) * weights.unsqueeze(1), dim = 3)
        new_features = torch.cat([new_features, features2], dim=1)
        new_features = self.conv(new_features.unsqueeze(3)).squeeze(3)
        return new_features

class FlowNet3D(nn.Module):
    def __init__(self):
        super(FlowNet3D, self).__init__()
        
        self.set_conv1 = SetConv(1024, 0.5, 16, 3, [32, 32, 64])
        self.set_conv2 = SetConv(256, 1.0, 16, 64, [64, 64, 128])
        self.flow_embedding = FlowEmbedding(64, 128, [128, 128, 128])
        self.set_conv3 = SetConv(64, 2.0, 8, 128, [128, 128, 256])
        self.set_conv4 = SetConv(16, 4.0, 8, 256, [256, 256, 512])
        self.set_upconv1 = SetUpConv(8, 512, 256, [], [256, 256])
        self.set_upconv2 = SetUpConv(8, 256, 256, [128, 128, 256], [256])
        self.set_upconv3 = SetUpConv(8, 256, 64, [128, 128, 256], [256])
        self.fp = FeaturePropagation(256, 3, [256, 256])
        self.classifier = nn.Sequential(
            nn.Conv1d(256, 128, 1, bias=True),
            nn.BatchNorm1d(128, eps=0.001),
            nn.ReLU(),
            nn.Conv1d(128, 3, 1, bias=True)
        )
         
    def forward(self, points1, points2, features1, features2):
        points1_1, features1_1 = self.set_conv1(points1, features1)
        points1_2, features1_2 = self.set_conv2(points1_1, features1_1)

        points2_1, features2_1 = self.set_conv1(points2, features2)
        points2_2, features2_2 = self.set_conv2(points2_1, features2_1)

        embedding = self.flow_embedding(points1_2, points2_2, features1_2, features2_2)
        
        points1_3, features1_3 = self.set_conv3(points1_2, embedding)
        points1_4, features1_4 = self.set_conv4(points1_3, features1_3)
        
        new_features1_3 = self.set_upconv1(points1_4, points1_3, features1_4, features1_3)
        new_features1_2 = self.set_upconv2(points1_3, points1_2, new_features1_3, torch.cat([features1_2, embedding], dim=1))
        new_features1_1 = self.set_upconv3(points1_2, points1_1, new_features1_2, features1_1)
        new_features1 = self.fp(points1_1, points1, new_features1_1, features1)

        flow = self.classifier(new_features1)
        
        return flow


In [ ]:
%%time

# data
test_set = TestDataset()

print('test set:', len(test_set))

# model
net = FlowNet3D().cuda()
net.load_state_dict(torch.load('./flownet3d.pytorch/models/net.pth'))
net.eval()

# statistics
loss_sum = 0
epe_sum = 0
acc050_sum = 0
acc010_sum = 0

with torch.no_grad():
    
    # for each mini-batch
    for points1, points2, flow in test_set:
            
        # to GPU
        points1 = points1.unsqueeze(0).cuda().contiguous()
        points2 = points2.unsqueeze(0).cuda().contiguous()
        flow = flow.unsqueeze(0).cuda().contiguous()
    
        # forward
        features1 = torch.zeros_like(points1)
        features2 = torch.zeros_like(points2)
        pred_flow = net(points1, points2, features1, features2)
        
        # statistics
        mask1 = torch.ones_like(features1)
        loss = criterion(pred_flow, flow, mask1)
        loss_sum += loss.item()
        epe, acc050, acc010 = error(pred_flow, flow, mask1)
        epe_sum += epe
        acc050_sum += acc050
        acc010_sum += acc010
        
print('mean loss:', loss_sum/len(test_set))
print('mean epe:', epe_sum/len(test_set))
print('mean acc050:', acc050_sum/len(test_set))
    
print('---')

test set: 150
mean loss: 0.022397985982242973
mean epe: 0.1567641403277715
mean acc050: 0.12694986979166667
---
CPU times: user 6.42 s, sys: 1.48 s, total: 7.9 s
Wall time: 7.9 s
